
# Assignment 3

This is a template notebook for Assignment 3.


## Install dependencies and initialization

In [ ]:
# install dependencies: 
!pip install pyyaml==5.1 pycocotools>=2.0.1
# !pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.6/index.html
!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu111/torch1.9/index.html

In [ ]:
!pwd # shows current directory
!ls  # shows all files in this directory
!nvidia-smi # shows the specs and the current status of the allocated GPU

In [ ]:
# import some common libraries
from google.colab.patches import cv2_imshow
from sklearn.metrics import jaccard_score
from PIL import Image, ImageDraw
from tqdm.notebook import tqdm
import pandas as pd
import numpy as np
import datetime
import random
import json
import cv2
import csv
import os

# import some common pytorch utilities
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
from torch.autograd import Variable
import torch.nn.functional as F
import torch.nn as nn
import torch

# import some common detectron2 utilities
import detectron2
from detectron2 import model_zoo
from detectron2.config import get_cfg
from detectron2.structures import BoxMode
from detectron2.engine import DefaultTrainer
from detectron2.engine import DefaultPredictor
from detectron2.utils.logger import setup_logger
from detectron2.utils.visualizer import ColorMode
from detectron2.utils.visualizer import Visualizer
from detectron2.data import build_detection_test_loader
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
setup_logger()

In [ ]:
# Make sure that GPU is available for your notebook. 
# Otherwise, you need to update the settungs in Runtime -> Change runtime type -> Hardware accelerator
torch.cuda.is_available()

In [ ]:
# You need to mount your google drive in order to load the data:
from google.colab import drive
drive.mount('/content/drive')
# Put all the corresponding data files in a data folder and put the data folder in a same directory with this notebook.
# Also create an output directory for your files such as the trained models and the output images.

In [ ]:
# Define the location of current directory, which should contain data/train, data/test, and data/train.json.
# TODO: approx 1 line
BASE_DIR = '/content/drive/My Drive/CMPT_CV_lab3'
OUTPUT_DIR = '{}/output'.format(BASE_DIR)
os.makedirs(OUTPUT_DIR, exist_ok=True)

## Part 1: Object Detection

### Data Loader

In [ ]:
'''
# This function should return a list of data samples in which each sample is a dictionary. 
# Make sure to select the correct bbox_mode for the data
# For the test data, you only have access to the images, therefore, the annotations should be empty.
# Other values could be obtained from the image files.
# param setname : "train", "test", or "val"
# TODO: approx 35 lines
'''
import math

def get_detection_data_from_disk(set_name):
  data_dirs = '{}/data'.format(BASE_DIR)

  imgs_anns = []

  if set_name != 'test':
    json_file = os.path.join(data_dirs, "train.json")
    with open(json_file) as f:
        imgs_anns = json.load(f)

  data_dict = {}

  start = 0
  end = len(imgs_anns)

  print('start is:',start)
  print('end is:',end)
  x= 0 
  for i in range(start, end):
    v = imgs_anns[i]
    if v['image_id'] not in data_dict:
      record = {}
      filename = os.path.join(f"{data_dirs}/train", v["file_name"])
      height, width = cv2.imread(filename).shape[:2]
      record["file_name"] = filename
      record["image_id"] = v['image_id']
      record["height"] = height
      record["width"] = width
      record['annotations'] = []
      record['segments_info'] = []
      data_dict[v['image_id']] = record

    annotation = {
        'bbox': v['bbox'],
        'bbox_mode': BoxMode.XYWH_ABS,
        'category_id': 0,
        'segmentation': v['segmentation'],
        'iscrowd': v['iscrowd']
    }
    data_dict[v['image_id']]['annotations'].append(annotation)

    segment = {
        'id': v['id'],
        'category_id': 0,
        'iscrowd': v['iscrowd']
    }
    data_dict[v['image_id']]['segments_info'].append(segment)

    x+=1
    if(x % 1000 == 0):
      print(x)

  idx = 0
  if len(imgs_anns) == 0:
    for file in os.listdir(f"{data_dirs}/{set_name}"):
      filename = os.path.join(f"{data_dirs}/{set_name}", file)
      height, width = cv2.imread(filename).shape[:2]
      record = {}
      record["file_name"] = filename
      record["image_id"] = idx
      record["height"] = height
      record["width"] = width
      record['annotations'] = []
      data_dict[idx] = record
      idx+=1

  dataset = list(data_dict.values())

  out_file = open(BASE_DIR + f"/data/dataset_{set_name}.json","w")
  json.dump(dataset, out_file)
  out_file.close()

  print('done')
  print(len(dataset))
  
  return dataset

def get_detection_data(set_name):
  if not os.path.isfile(BASE_DIR + f"/data/dataset_{set_name}.json"):
    return get_detection_data_from_disk(set_name)
 
  f = open(BASE_DIR + f"/data/dataset_{set_name}.json")
  data = json.load(f)
  print(f'dataset_{set_name} loaded from file!')
  return data

In [ ]:
'''
# Remember to add your dataset to DatasetCatalog and MetadataCatalog
# Consdier "data_detection_train" and "data_detection_test" for registration
# You can also add an optional "data_detection_val" for your validation by spliting the training data
# TODO: approx 5 lines
'''
from sklearn.model_selection import train_test_split

# clear
DatasetCatalog.clear()
MetadataCatalog.clear()

for d in ["train", "test", "val"]:
    DatasetCatalog.register("data_detection_" + d, lambda d=d: get_detection_data(d))
    MetadataCatalog.get("data_detection_" + d).set(thing_classes=["plane"])
metadata_train = MetadataCatalog.get("data_detection_train")
metadata_test = MetadataCatalog.get("data_detection_test")

In [ ]:
'''
# Visualize some samples using Visualizer to make sure that the function works correctly
# TODO: approx 5 lines
'''
for d in random.sample(dataset_dicts_train, 3):
    img = cv2.imread(d["file_name"])
    visualizer = Visualizer(img[:, :, ::-1], metadata=metadata_train, scale=0.5)
    # visualizer = Visualizer(img[:, :, ::-1], scale=0.5)
    out = visualizer.draw_dataset_dict(d)
    cv2_imshow(out.get_image()[:, :, ::-1])


### Set Configs

In [ ]:
'''
# Set the configs for the detection part in here.
# TODO: approx 15 lines
'''
'''
# THIS IS THE BAD CONFIG

cfg = get_cfg()
cfg.OUTPUT_DIR = "{}/output/badconfig/".format(BASE_DIR)

cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_101_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("data_detection_train",)
cfg.DATASETS.TEST = ()
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_R_101_FPN_3x.yaml")  # Let training initialize from model zoo
cfg.SOLVER.BATCH_SIZE_PER_IMAGE = 128

cfg.SOLVER.IMS_PER_BATCH = 2  # batch size
cfg.SOLVER.BASE_LR = 0.00025  # pick a good LR
cfg.SOLVER.MAX_ITER = 500  # and a good number of iterations
cfg.SOLVER.STEPS = []        # do not decay learning rate

### Training

In [ ]:
'''
# Create a DefaultTrainer using the above config and train the model
# TODO: approx 5 lines
'''
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = DefaultTrainer(cfg) 
trainer.resume_or_load(resume=False)
trainer.train()

In [ ]:
# Look at training curves in tensorboard:
%load_ext tensorboard
%tensorboard --logdir "/content/drive/My Drive/CMPT_CV_lab3/output"

### Evaluation and Visualization

In [ ]:
'''
# After training the model, you need to update cfg.MODEL.WEIGHTS
# Define a DefaultPredictor
'''
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")  # path to the model we just trained
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7   # set a custom testing threshold
predictor = DefaultPredictor(cfg)

In [ ]:
'''
# Visualize the output for 3 random test samples
# TODO: approx 10 lines
'''
dataset_dicts_test = get_detection_data("test")

for d in random.sample(dataset_dicts_test, 3):    
    im = cv2.imread(d["file_name"])
    outputs = predictor(im)  # format is documented at https://detectron2.readthedocs.io/tutorials/models.html#model-output-format
    v = Visualizer(im[:, :, ::-1],
                   metadata=metadata_train, 
                   scale=0.5, 
                   instance_mode=ColorMode.IMAGE_BW   # remove the colors of unsegmented pixels. This option is only available for segmentation models
    )
    out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    cv2_imshow(out.get_image()[:, :, ::-1])

In [ ]:
'''
# Use COCOEvaluator and build_detection_train_loader
# You can save the output predictions using inference_on_dataset
# TODO: approx 5 lines
'''
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader
evaluator = COCOEvaluator("data_detection_train", output_dir=OUTPUT_DIR)


In [ ]:
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.6  # set threshold for this model

val_loader = build_detection_test_loader(cfg, "data_detection_train")
print(inference_on_dataset(predictor.model, val_loader, evaluator))

### Improvements

For this part, you can bring any improvement which you have by adding new input parameters to the previous functions or defining new functions and variables.

In [ ]:
'''
# Bring any changes and updates regarding the improvement in here
'''
# THIS IS THE FINAL CONFIG

cfg = get_cfg()
cfg.OUTPUT_DIR = "{}/output/".format(BASE_DIR)

cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("data_detection_train",)
cfg.DATASETS.TEST = ()
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml")  # Let training initialize from model zoo
cfg.SOLVER.BATCH_SIZE_PER_IMAGE = 512
cfg.SOLVER.IMS_PER_BATCH = 4  # batch size
cfg.SOLVER.BASE_LR = 0.0005  # pick a good LR
cfg.SOLVER.MAX_ITER = 1000  # and a good number of iterations
cfg.SOLVER.STEPS = []        # do not decay learning rate

Visual 3 images selected from the test set: for visualisation of a sample for qualitative comparison.

In [ ]:
sample_idx = [1, 10, 45]
for idx in sample_idx:    
    d = dataset_dicts_test[idx]
    im = cv2.imread(d["file_name"])
    outputs = predictor(im)  # format is documented at https://detectron2.readthedocs.io/tutorials/models.html#model-output-format
    v = Visualizer(im[:, :, ::-1],
                   metadata=metadata_train, 
                   scale=0.5, 
                   instance_mode=ColorMode.IMAGE_BW   # remove the colors of unsegmented pixels. This option is only available for segmentation models
    )
    out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    cv2_imshow(out.get_image()[:, :, ::-1])

## Part 2: Semantic Segmentation

### Get instance sample


In [ ]:
'''
# Write a function that returns the cropped image and corresponding mask regarding the target bounding box
# idx is the index of the target bbox in the data
# high-resolution image could be passed or could be load from data['file_name']
# You can use the mask attribute of detectron2.utils.visualizer.GenericMask 
#     to convert the segmentation annotations to binary masks
# TODO: approx 10 lines
'''
from detectron2.utils.visualizer import GenericMask
from google.colab.patches import cv2_imshow


def get_instance_sample(data, idx, img=None, only='', resize=True):
  mask = None

  x, y, w, h = np.array(data['annotations'][idx]['bbox']).astype(int)

  # resize size
  l1 = 128

  if only != 'img':
    # getting mask 
    g_mask = GenericMask(data['annotations'][idx]['segmentation'], data['height'], data['width'])
    obj_mask = g_mask.mask
    mask = obj_mask[y:y+h, x:x+w]
    if resize:
      mask = cv2.resize(obj_mask, (l1,l1))

  if only != 'mask':
    # getting image
    im = None
    if img is not None:
      im = Image.fromarray(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    else:
      im = Image.open(data['file_name']).convert('RGB')
    img = im.crop((x, y, x+w, y+h))
    if resize:
      img = img.resize((l1,l1))

  return img, mask

# test
# my_data_list = DatasetCatalog.get("data_detection_{}".format('train'))
# get_instance_sample(my_data_list[1], 2)

### Moving images to h5

In [ ]:
def file_instance_map():
  train_labels = DatasetCatalog.get("data_detection_{}".format('train'))

  # use instance map to preserve order from the template code
  instance_map = []
  for i, d in enumerate(train_labels):
          for j in range(len(d['annotations'])):
            instance_map.append([i,j])

# file_instance_map()

In [ ]:
import h5py
data_dirs = '{}/data'.format(BASE_DIR)
hdf5_path = f"{data_dirs}/planes.hdf5"  # path for .hdf5 file
train_img_path = f"{data_dirs}/train"

def create_h5_dataset():
  num_bbox = len(instance_map)

  train_X_shape = (num_bbox, 128, 128, 3)
  train_Y_shape = (num_bbox, 128, 128, 1)

  # open a hdf5 file
  f = h5py.File(hdf5_path, mode='w')

  f.create_dataset("train_X", train_X_shape, np.uint8)
  f.create_dataset("train_Y", train_Y_shape, np.uint8)

  # f.create_dataset("train_labels", (len(train_labels),), np.uint8)
  # f["train_labels"][...] = train_labels

  i = 0
  for idx in instance_map:

    if i % 10 == 0 and i > 1:
      print(f'Serializing train data: {i}')
      break

    if torch.is_tensor(idx):
        idx = idx.tolist()
    data = train_labels[idx[0]]
    img, mask = get_instance_sample(data, idx[1])

    f['train_X'][i, ...] = img
    f['train_Y'][i, ...] = mask

    i+=1

  f.close()

# create_h5_dataset()

### Data Loader

In [ ]:
'''
# We have provided a template data loader for your segmentation training
# You need to complete the __getitem__() function before running the code
# You may also need to add data augmentation or normalization in here
'''

class PlaneDataset(Dataset):
  def __init__(self):
      self.transforms = transforms.Compose([
          # transforms.RandomHorizontalFlip(0.5),
          # transforms.RandomCrop(32),
          transforms.ToTensor(),
          transforms.Normalize(mean=0,
                                std=1)
          # transforms.Normalize(mean=[0.485, 0.456, 0.406],
          #                       std=[0.229, 0.224, 0.225])
      ])
      self.file = h5py.File(hdf5_path, mode='r')
      print('loaded file\n')
      print(self.file)

  '''
  # you can change the value of length to a small number like 10 for debugging of your training procedure and overfeating
  # make sure to use the correct length for the final training
  '''
  def __len__(self):
      return self.file['train_X'].shape[0]

  def numpy_to_tensor(self, img, mask):
    if self.transforms is not None:
        img = self.transforms(img)
    img = torch.tensor(img, dtype=torch.float)
    mask = torch.tensor(mask, dtype=torch.float)
    return img, mask

  '''
  # Complete this part by using get_instance_sample function
  # make sure to resize the img and mask to a fixed size (for example 128*128)
  # you can use "interpolate" function of pytorch or "numpy.resize"
  # TODO: 5 lines
  '''
  def __getitem__(self, idx):
    img = self.file['train_X'][idx, ...]
    mask = self.file['train_Y'][idx, ...]
    
    img = Image.fromarray(img)
    mask = np.array(mask).reshape((1,128,128))


    return self.numpy_to_tensor(img, mask)

def get_plane_dataset(set_name='train', batch_size=2):
    # my_data_list = DatasetCatalog.get("data_detection_{}".format(set_name))
    dataset = PlaneDataset()
    loader = DataLoader(dataset, batch_size=batch_size, num_workers=4,
                                              pin_memory=True, shuffle=True)
    return loader, dataset

### Network

In [ ]:
'''
# convolution module as a template layer consists of conv2d layer, batch normalization, and relu activation
'''
class conv(nn.Module):
    def __init__(self, in_ch, out_ch, activation=True):
        super(conv, self).__init__()
        if(activation):
          self.layer = nn.Sequential(
             nn.Conv2d(in_ch, out_ch, 3, padding=1),
             nn.BatchNorm2d(out_ch),
             nn.ReLU(inplace=True)
          )
        else:
          self.layer = nn.Sequential(
             nn.Conv2d(in_ch, out_ch, 3, padding=1)  
             )

    def forward(self, x):
        x = self.layer(x)
        return x

'''
# downsampling module equal to a conv module followed by a max-pool layer
'''
class down(nn.Module):
    def __init__(self, in_ch, out_ch):
        super(down, self).__init__()
        self.layer = nn.Sequential(
            conv(in_ch, out_ch),
            nn.MaxPool2d(2)
            )

    def forward(self, x):
        x = self.layer(x)
        return x

'''
# upsampling module equal to a upsample function followed by a conv module
'''
class up(nn.Module):
    def __init__(self, in_ch, out_ch, bilinear=False):
        super(up, self).__init__()
        if bilinear:
            self.up = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
        else:
            self.up = nn.ConvTranspose2d(in_ch, in_ch, 2, stride=2)

        self.conv = conv(in_ch, out_ch)

    def forward(self, x):
        y = self.up(x)
        y = self.conv(y)
        return y

'''
# the main model which you need to complete by using above modules.
# you can also modify the above modules in order to improve your results.
'''
class MyModel(nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()
        
        # Encoder
        
        self.conv1 = conv(3, 64)
        self.down1 = down(64, 64)
        # Output size = 128//2 x 128//2 = 64 x 64
        self.conv2 = conv(64, 128)
        self.down2 = down(128, 128)
        # Output size = 64//2 x 64//2 = 32 x 32
        self.conv3 = conv(128, 256)
        self.down3 = down(256, 256)
        # Output size = 32//2 x 32//2 = 16 x 16
        self.conv4 = conv(256, 512)
        self.conv5 = conv(512, 512)
        
        # Decoder

        self.up1 = up(512, 256)
        # Output size = 16*2 x 16*2 = 32 x 32
        self.conv6 = conv(256, 256)
        self.up2 = up(256, 128)
        # Output size = 32*2 x 32*2 = 64 x 64
        self.conv7 = conv(128, 128)
        self.up3 = up(128, 64)
        # Output size = 64*2 x 64*2 = 128 x 128
        self.conv8 = conv(64, 64)
        self.output_conv = conv(64, 1, False) # ReLu activation is removed to keep the logits for the loss function
        

    def forward(self, input):
      y = self.conv1(input)
      y = self.down1(y)
      y = self.conv2(y)
      y = self.down2(y)
      y = self.conv3(y)
      y = self.down3(y)
      y = self.conv4(y)
      y = self.conv5(y)
      y = self.up1(y)
      y = self.conv6(y)
      y = self.up2(y)
      y = self.conv7(y)
      y = self.up3(y)
      y = self.conv8(y)
      output = self.output_conv(y)
      return output

### Training

In [ ]:
# The following is a basic training procedure to train the network
# You need to update the code to get the best performance
# TODO: approx ? lines
'''
'''
# Set the hyperparameters
num_epochs = 5
# num_epochs = 0 # to stop re-training when running notebook
batch_size = 4
learning_rate = 0.02
weight_decay = 1e-5

model = MyModel() # initialize the model
model = model.cuda() # move the model to GPU
loader, _ = get_plane_dataset('train', batch_size) # initialize data_loader
crit = nn.BCEWithLogitsLoss() # Define the loss function
optim = torch.optim.SGD(model.parameters(), lr=learning_rate, weight_decay=weight_decay) # Initialize the optimizer as SGD

# start the training procedure
for epoch in range(num_epochs):
  total_loss = 0
  for (img, mask) in tqdm(loader):
    img = torch.tensor(img, device=torch.device('cuda'), requires_grad = True)
    mask = torch.tensor(mask, device=torch.device('cuda'), requires_grad = True)
    pred = model(img)
    loss = crit(pred, mask)
    optim.zero_grad()
    loss.backward()
    optim.step()
    total_loss += loss.cpu().data
  print("Epoch: {}, Loss: {}".format(epoch, total_loss/len(loader)))
  torch.save(model.state_dict(), '{}/output/{}_segmentation_model.pth'.format(BASE_DIR, epoch))

'''
# Saving the final model
'''
torch.save(model.state_dict(), '{}/output/final_segmentation_model.pth'.format(BASE_DIR))

### Evaluation and Visualization

In [ ]:
'''
# Before starting the evaluation, you need to set the model mode to eval
# You may load the trained model again, in case if you want to continue your code later
# TODO: approx 15 lines
'''
import matplotlib.pyplot as plt

batch_size = 8
model = MyModel().cuda()
model.load_state_dict(torch.load('{}/output/final_segmentation_model.pth'.format(BASE_DIR)))
model = model.eval() # changing the model to evaluation mode will fix the bachnorm layers
loader, dataset = get_plane_dataset('train', batch_size)

THRESHOLD = -0.5 # used to threshold predicted mask

total_iou = 0
i = 0 #del
for (img, mask) in tqdm(loader):
  with torch.no_grad():
    img = img.cuda()
    mask = mask.cuda()
    pred = model(img)

    for b in range(batch_size):
      # conv to cv2
      a =  pred[b].detach().cpu().squeeze().numpy()
      b =  mask[b].detach().cpu().squeeze().numpy()

      _, a_thresh = cv2.threshold(a, THRESHOLD, 1.0,cv2.THRESH_BINARY)

      # calc IoU
      inter = cv2.bitwise_and(a_thresh, b)
      union = cv2.bitwise_or(a_thresh, b)
      inter_c = cv2.countNonZero(inter)
      union_c = cv2.countNonZero(union)
      iou = inter_c/union_c
      total_iou += iou

      if i>0 and i % 1000 == 0:
        print(f"iter {i}, curr mean IoU: {total_iou/(i+1)}")
      i+=1
    
      # visualization
      # a_n = cv2.normalize(src=a_thresh, dst=None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8U)
      # b_n = cv2.normalize(src=b, dst=None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8U)
      # inter_n = cv2.normalize(src=inter, dst=None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8U)
      # cv2_imshow(a_n)
      # cv2_imshow(b_n)
    
mean_iou = total_iou/len(dataset)
print("\n #images: {}, Mean IoU: {}".format(len(dataset), mean_iou))

In [ ]:
'''
# Visualize 3 sample outputs
# TODO: approx 5 lines
'''
trans = transforms.ToTensor()

for file in os.listdir(f"{data_dirs}/test_cropped"):
    filename = os.path.join(f"{data_dirs}//test_cropped", file)
    im = Image.open(filename).resize((128,128))
    img = torch.tensor(trans(im), dtype=torch.float)
    img = img.unsqueeze(0)
    predt = model(img.cuda())
    a =  predt[0].detach().cpu().squeeze().numpy()

    cv_im = np.array(im) 
    # Convert RGB to BGR 
    cv_im = cv_im[:, :, ::-1].copy() 

    _, a_thresh = cv2.threshold(a, THRESHOLD, 1.0,cv2.THRESH_BINARY)
    a_n = cv2.normalize(src=a_thresh, dst=None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8U)
    cv2_imshow(cv_im)
    cv2_imshow(a_n)

## Part 3: Instance Segmentation

In this part, you need to obtain the instance segmentation results for the test data by using the trained segmentation model in the previous part and the detection model in Part 1.

### Get Prediction

In [ ]:
'''
# Define a new function to obtain the prediction mask by passing a sample data
# For this part, you need to use all the previous parts (predictor, get_instance_sample, data preprocessings, etc)
# It is better to keep everything (as well as the output of this funcion) on gpu as tensors to speed up the operations.
# pred_mask is the instance segmentation result and should have different values for different planes.
# TODO: approx 35 lines
'''
dataset_dicts_test = get_detection_data("test") # run this line once
dataset_dicts_train = get_detection_data("train") # run this line once

def fill_pred_data(inp):
  data = {
    'annotations': [],
    'height': inp['instances'].image_size[0],
    'width': inp['instances'].image_size[1],
  }
  bboxes = inp['instances'].pred_boxes.tensor.cpu().numpy()
  for box in bboxes:
    # convert to XYWH
    b_dict = {'bbox': [box[0], box[1], box[2]-box[0], box[3]-box[1]]}
    data['annotations'].append(b_dict)
  return data

# used to convert img to tensor
trans = transforms.ToTensor()

def get_prediction_mask(data):
  im = cv2.imread(data["file_name"])
  # create gt_mask
  gt_mask = np.zeros((data['height'],data['width']), np.float32)
  for i in range(len(data['annotations'])):
    _, mask = get_instance_sample(data, i, im, only='mask', resize=False)
    x, y, w, h = np.array(data['annotations'][i]['bbox']).astype(int)
    gt_mask[y:y+h, x:x+w] = mask*(i+1)

  # detect planes and bounding boxes
  outputs = predictor(im)  # format is documented at https://detectron2.readthedocs.io/tutorials/models.html#model-output-format
  pred_data = fill_pred_data(outputs)

  # generate predicted pred_mask
  pred_mask = np.zeros((data['height'],data['width']), np.float32)
  for i in range(len(pred_data['annotations'])):
    img, _ = get_instance_sample(pred_data, i, im, only='img', resize=True)
    x, y, w, h = np.array(pred_data['annotations'][i]['bbox']).astype(int)
    img = torch.tensor(trans(img), dtype=torch.float)
    img = img.unsqueeze(0)
    predt = model(img.cuda())
    a =  predt[0].detach().cpu().squeeze().numpy()
    mask = cv2.resize(a, (w,h))
    _, mask = cv2.threshold(mask, THRESHOLD, i+1 ,cv2.THRESH_BINARY)
    pred_mask[y:y+h, x:x+w] = mask
  
  iou=0
  if len(data['annotations']) > 0:
    inter = cv2.countNonZero(cv2.bitwise_and(gt_mask, pred_mask))
    union = cv2.countNonZero(cv2.bitwise_or(gt_mask, pred_mask))
    iou = inter/float(union)
  
  return im, gt_mask, trans(pred_mask).cuda(), iou # gt_mask could be all zero when the ground truth is not given.

# get_prediction_mask(dataset_dicts_train[2]);
# get_prediction_mask(dataset_dicts_test[0]);

### Visualization and Submission

In [ ]:
'''
# Visualise the output prediction as well as the GT Mask and Input image for a sample input
# TODO: approx 10 lines
'''
data_list_train = DatasetCatalog.get("data_detection_{}".format('train'))
data_list_test = DatasetCatalog.get("data_detection_{}".format('test'))

samples = [data_list_train[0], data_list_train[1], data_list_train[2]]

# for sample in samples:
for d in random.sample(data_list_train, 3):    
  img, gt_mask, pred_mask, _ = get_prediction_mask(d);

  pred_mask = pred_mask[0].detach().cpu().squeeze().numpy()

  height, width, _ = img.shape

  img_n = cv2.normalize(src=img, dst=None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8U)
  img_n = cv2.resize(img_n, (width//2, height//2))
  cv2_imshow(img_n)

  pred_rgb = np.zeros((height,width,3), np.float32)
  gt_rgb = np.zeros((height,width,3), np.float32)
  num_instances = max(int(np.max(gt_mask)), int(np.max(pred_mask)))
  ins_to_color = []
  for i in range(num_instances):
    color = list(np.random.choice(range(100,256), size=3))
    ins_to_color.append(color)
    pred_rgb[np.where(pred_mask == i+1)] = color
    gt_rgb[np.where(gt_mask == i+1)] = color


  if cv2.countNonZero(gt_mask):
    gt_rgb = cv2.normalize(src=gt_rgb, dst=None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8U)
    gt_rgb = cv2.resize(gt_rgb, (width//2, height//2))
    print("ground truth:")
    cv2_imshow(gt_rgb)

  pred_rgb = cv2.normalize(src=pred_rgb, dst=None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8U)
  pred_rgb = cv2.resize(pred_rgb, (width//2, height//2))
  print("prediction:")
  cv2_imshow(pred_rgb)

  inter = cv2.countNonZero(cv2.bitwise_and(gt_mask, pred_mask))
  union = cv2.countNonZero(cv2.bitwise_or(gt_mask, pred_mask))

  iou = inter/float(union)
  print('IoU is', iou)



In [ ]:
'''
# ref: https://www.kaggle.com/rakhlin/fast-run-length-encoding-python
# https://www.kaggle.com/c/airbus-ship-detection/overview/evaluation
'''
def rle_encoding(x):
    '''
    x: pytorch tensor on gpu, 1 - mask, 0 - background
    Returns run length as list
    '''
    dots = torch.where(torch.flatten(x.long())==1)[0]
    if(len(dots)==0):
      return []
    inds = torch.where(dots[1:]!=dots[:-1]+1)[0]+1
    inds = torch.cat((torch.tensor([0], device=torch.device('cuda'), dtype=torch.long), inds))
    tmpdots = dots[inds]
    inds = torch.cat((inds, torch.tensor([len(dots)], device=torch.device('cuda'))))
    inds = inds[1:] - inds[:-1]
    runs = torch.cat((tmpdots, inds)).reshape((2,-1))
    runs = torch.flatten(torch.transpose(runs, 0, 1)).cpu().data.numpy()
    return ' '.join([str(i) for i in runs])

In [ ]:

# You need to upload the csv file on kaggle
# The speed of your code in the previous parts highly affects the running time of this part
'''
'''

preddic = {"ImageId": [], "EncodedPixels": []}

'''
# Writing the predictions of the training set
'''

total_iou = 0
data_list_train = DatasetCatalog.get("data_detection_{}".format('train'))
for i in tqdm(range(len(data_list_train)), position=0, leave=True):
  sample = data_list_train[i]
  sample['image_id'] = sample['file_name'].split("/")[-1][:-4]
  img, true_mask, pred_mask, iou = get_prediction_mask(sample)
  
  total_iou += iou
  if i % 10 == 0:
    print(f'iter {i}: avg iou: {total_iou/(i+1)}')
  
  inds = torch.unique(pred_mask)
  if(len(inds)==1):
    preddic['ImageId'].append(sample['image_id'])
    preddic['EncodedPixels'].append([])
  else:
    for index in inds:
      if(index == 0):
        continue
      tmp_mask = (pred_mask==index)
      encPix = rle_encoding(tmp_mask)
      preddic['ImageId'].append(sample['image_id'])
      preddic['EncodedPixels'].append(encPix)

print('avg iou:', total_iou/len(data_list_train))

'''
# Writing the predictions of the test set
'''

data_list_test = DatasetCatalog.get("data_detection_{}".format('test'))
for i in tqdm(range(len(data_list_test)), position=0, leave=True):
  sample = data_list_test[i]
  sample['image_id'] = sample['file_name'].split("/")[-1][:-4]
  img, true_mask, pred_mask, _ = get_prediction_mask(sample)
  inds = torch.unique(pred_mask)
  if(len(inds)==1):
    preddic['ImageId'].append(sample['image_id'])
    preddic['EncodedPixels'].append([])
  else:
    for j, index in enumerate(inds):
      if(index == 0):
        continue
      tmp_mask = (pred_mask==index).double()
      encPix = rle_encoding(tmp_mask)
      preddic['ImageId'].append(sample['image_id'])
      preddic['EncodedPixels'].append(encPix)

pred_file = open("{}/pred.csv".format(BASE_DIR), 'w')
pd.DataFrame(preddic).to_csv(pred_file, index=False)
pred_file.close()



## Part 4: Mask R-CNN

For this part you need to follow a same procedure to part 2 with the configs of Mask R-CNN, other parts are generally the same as part 2.

### Config and Training

In [ ]:
from detectron2.engine import DefaultTrainer

cfg = get_cfg()
cfg.OUTPUT_DIR = "{}/output/mask_rcnn/".format(BASE_DIR)
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("data_detection_train",)
cfg.DATASETS.TEST = ()
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")  # Let training initialize from model zoo
cfg.SOLVER.IMS_PER_BATCH = 4  # batch size
cfg.SOLVER.BASE_LR = 0.0005  # pick a good LR
cfg.SOLVER.MAX_ITER = 500    # 300 iterations seems good enough for this toy dataset; you will need to train longer for a practical dataset
cfg.SOLVER.STEPS = []        # do not decay learning rate
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 512
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1 
# NOTE: this config means the number of classes, but a few popular unofficial tutorials incorrect uses num_classes+1 here.

os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = DefaultTrainer(cfg) 
trainer.resume_or_load(resume=False)
trainer.train()

### Visualization

In [ ]:
# Inference should use the config with parameters that are used in training
# cfg now already contains everything we've set previously. We changed it a little bit for inference:

cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")  # path to the model we just trained
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7 # set threshold for this model
predictor = DefaultPredictor(cfg)

In [ ]:
from detectron2.utils.visualizer import ColorMode
dataset_dicts = get_detection_data("train")

for d in random.sample(dataset_dicts, 3):    
    im = cv2.imread(d["file_name"])
    outputs = predictor(im)  # format is documented at https://detectron2.readthedocs.io/tutorials/models.html#model-output-format
    v = Visualizer(im[:, :, ::-1],
                   metadata=metadata_train, 
                   scale=0.5, 
                   instance_mode=ColorMode.SEGMENTATION   # remove the colors of unsegmented pixels. This option is only available for segmentation models
    )
    out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    cv2_imshow(out.get_image()[:, :, ::-1])


### Evaluation



In [ ]:
'''
# Use COCOEvaluator and build_detection_train_loader
# You can save the output predictions using inference_on_dataset
# TODO: approx 5 lines
'''
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader
evaluator = COCOEvaluator("data_detection_train", output_dir=OUTPUT_DIR)

val_loader = build_detection_test_loader(cfg, "data_detection_train")
print(inference_on_dataset(predictor.model, val_loader, evaluator))